# Experian Gross to Net Income Task 1 - Group 8

### You should submit this notebook, the salary_holdout.csv with the populated net to gross incomes, and a word/pdf doc for task 2. Submit the csv according to the format it is received, with the annual_gross_income column filled with appropriate values.

----------
Written by Lucas Murphy and Emrah Selcuk 2022

## Purpose of this task

By the end of this task you should:
    
    - have an in depth understanding of tax system used in the UK
    - be able to reverse net back into gross income
    - understand some of the challenges faced when inferring gross income from net

### Setup

Read in CSV of incomes...

In [1]:
import pandas as pd
import openpyxl

df = pd.read_csv('salary_test_24.csv')
df

FileNotFoundError: [Errno 2] No such file or directory: 'salary_test_24.csv'

### Data Inspection

It's worthwhile to inspect your data beforehand to ensure it is as expected.

In [2]:
salaries = pd.read_csv('salary_holdout_24.csv')
salaries

,location,monthly_net_income,annual_gross_income
0,rou,3041.79,0.0
1,rou,2466.31,0.0
2,rou,2862.15,0.0
3,sco,2919.71,0.0
4,rou,1601.49,0.0
...,...,...,...
4995,rou,2367.88,0.0
4996,rou,1736.65,0.0
4997,rou,1849.65,0.0
4998,rou,3207.28,0.0


In [156]:
#This function uses the test data to calculate the net monthly salaries from the gross annual salaries
def applying_tax(salary_df):
    net_monthly_salaries_sco = [] #Initialising lists to store the net monthly salaries once they have been calculated
    net_monthly_salaries_rou = []

    location_salaries = salary_df.groupby('location') #Grouping by location
    rou_data = location_salaries.get_group('rou') #Creating dataframe with rest of UK salaries
    scotland_data = location_salaries.get_group('sco') #Creating dataframe with Scotland salaries

    # Extract annual gross incomes as a list
    gross_annual_incomes_Sco = scotland_data['annual_gross_income'].values.tolist()
    gross_annual_incomes_Rou = rou_data['annual_gross_income'].values.tolist()


    #For Scottish Data
    for gross_annual_income in gross_annual_incomes_Sco: #Looping through all annual incomes
        deductions = 0  # Reset deductions for each salary

        # Income Tax:
        taxable_income = gross_annual_income #Setting annual gross income as taxable income as personal allowance is taken off in first band
        min_income_predicate = [0, 12570, 14876, 26561, 43662, 75000, 125140] #Setting list of all minimumn values to enter each tax band
        max_income_predicate = [12571, 14877, 26562, 43663, 75001, 125141, float('inf')] #Setting list of all max values of each tax band, infinte value is used to compare when the band has no max value set
        tax_rate = [0, 0.19, 0.2, 0.21, 0.42, 0.45, 0.48] #Setting tax rates for each tax band

        for i in range(7): #Looping through the number of tax bands
            if taxable_income > min_income_predicate[i]: #Judging if income in high enough to enter tax band
                
                if taxable_income > max_income_predicate[i]: #Checking if income is higher than the max value of tax band
                    deductions += tax_rate[i] * (max_income_predicate[i] - min_income_predicate[i]) #deducitng the tax rate applied the the eligible amount of salary for that band
                else:
                    deductions += tax_rate[i] * (taxable_income - min_income_predicate[i]) #When the income maxes out within the band and wont go any bands higher, the total income is used as the max value to apply tax rate as it falls within the band

        # National Insurance:
        taxable_income = gross_annual_income #All of these have the same logic as the income tax application except the values change 
        min_income_predicate = [0, 11904, 50270]
        max_income_predicate = [11905, 50271, float('inf')]
        tax_rate = [0, 0.08, 0.02]

        for i in range(3):  #Iterates a different amount of times as income tax to match the number of National Insurance bands
            if taxable_income > min_income_predicate[i]:
                
                if taxable_income > max_income_predicate[i]:
                    deductions += tax_rate[i] * (max_income_predicate[i] - min_income_predicate[i])
                else:
                    deductions += tax_rate[i] * (taxable_income - min_income_predicate[i])

        # Calculate net salary
        net_annual_salary = gross_annual_income - deductions #Calculates net annual salary by subtracting the final running total of deductions previously calculated
        net_monthly_salary = round(net_annual_salary / 12,2) #Dividing by 12 to get monthly net salary while also rounding the value to 2 decimal places
        net_monthly_salaries_sco.append(net_monthly_salary) #Appending all of the net monthly salaries onto a list, specific for Scotland 

    
    #For Rest of the UK Data - Will do all the same code again but the tax bands and tax rates are different, which in turn changes the number of times the loops are repeated
    for gross_annual_income in gross_annual_incomes_Rou:
        deductions = 0  # Reset deductions for each salary

        # Income Tax:
        taxable_income = gross_annual_income
        min_income_predicate = [0, 12570, 50270, 125140]
        max_income_predicate = [12571, 50271, 125141, float('inf')]
        tax_rate = [0, 0.2, 0.4, 0.45]

        for i in range(4):
            if taxable_income > min_income_predicate[i]:
                
                if taxable_income > max_income_predicate[i]:
                    deductions += tax_rate[i] * (max_income_predicate[i] - min_income_predicate[i])
                else:
                    deductions += tax_rate[i] * (taxable_income - min_income_predicate[i])

        # National Insurance: - The national insurance is the exact same as in Scotland therefore this code is identical
        taxable_income = gross_annual_income
        min_income_predicate = [0, 11904, 50270]
        max_income_predicate = [11905, 50271, float('inf')]
        tax_rate = [0, 0.08, 0.02]

        for i in range(3):  
            if taxable_income > min_income_predicate[i]:
                
                if taxable_income > max_income_predicate[i]:
                    deductions += tax_rate[i] * (max_income_predicate[i] - min_income_predicate[i])
                else:
                    deductions += tax_rate[i] * (taxable_income - min_income_predicate[i])

        # Calculate net salary - Same calculations as for Scotland except this data is being stored in a specific Rest of UK list to seperate the data from Scottish Salaries
        net_annual_salary = gross_annual_income - deductions
        net_monthly_salary = round(net_annual_salary / 12,2)
        net_monthly_salaries_rou.append(net_monthly_salary)



    #using the results of the Socttish data to create a dataframe with the net monthly salaries and the gross annual salaries that were given
    sottish_gross_to_net_incomes = {
        'Monthly Net Income': net_monthly_salaries_sco,
        'Annual Gross Income': gross_annual_incomes_Sco
    }
    sottish_gross_to_net_incomes_df = pd.DataFrame(sottish_gross_to_net_incomes)

    #Same with Rest of UK salaries
    rou_gross_to_net_incomes = {
        'Monthly Net Income': net_monthly_salaries_rou,
        'Annual Gross Income': gross_annual_incomes_Rou
    }
    rou_gross_to_net_incomes_df = pd.DataFrame(rou_gross_to_net_incomes)

    return sottish_gross_to_net_incomes_df, rou_gross_to_net_incomes_df #Returning both dataframes
        
    
 


#The following functions are helper functions to assist with going from the monthly net income to the annual gross income
#These functions, both specified for either Scotland ot ROU, copy the code that takes the annual gross income and calculates the monthly net income
#Although instead of taking in a dataframe of data and applying the function to all entires in a dataframe, this function only takes in a single gross annual salary given to it by the main function to calculate whats that gross value's monthly net income would be
#This net monthly income can then be compared to what the actual net monthly income should be to determine how far of the calculated gross value is. This is then used to adjust the calcualted gross income
#Function specifically for Scotland:
def applying_scottish_tax(salary):
    deductions = 0  # Reset deductions for each salary

        # Income Tax:
    taxable_income = salary #Setting annual gross income as taxable income as personal allowance is taken off in first band
    min_income_predicate = [0, 12570, 14876, 26561, 43662, 75000, 125140] #Setting list of all minimumn values to enter each tax band
    max_income_predicate = [12571, 14877, 26562, 43663, 75001, 125141, float('inf')] #Setting list of all max values of each tax band, infinte value is used to compare when the band has no max value set
    tax_rate = [0, 0.19, 0.2, 0.21, 0.42, 0.45, 0.48] #Setting tax rates for each tax band

    for i in range(7): #Looping through the number of tax bands
        if taxable_income > min_income_predicate[i]: #Judging if income in high enough to enter tax band
                
            if taxable_income > max_income_predicate[i]: #Checking if income is higher than the max value of tax band
                deductions += tax_rate[i] * (max_income_predicate[i] - min_income_predicate[i]) #deducitng the tax rate applied the the eligible amount of salary for that band
            else:
                deductions += tax_rate[i] * (taxable_income - min_income_predicate[i]) #When the income maxes out within the band and wont go any bands higher, the total income is used as the max value to apply tax rate as it falls within the band

        # National Insurance:
    taxable_income = salary #All of these have the same logic as the income tax application except the values change 
    min_income_predicate = [0, 11904, 50270]
    max_income_predicate = [11905, 50271, float('inf')]
    tax_rate = [0, 0.08, 0.02]

    for i in range(3):  #Iterates a different amount of times as income tax to match the number of National Insurance bands
        if taxable_income > min_income_predicate[i]:
                
            if taxable_income > max_income_predicate[i]:
                deductions += tax_rate[i] * (max_income_predicate[i] - min_income_predicate[i])
            else:
                deductions += tax_rate[i] * (taxable_income - min_income_predicate[i])

        # Calculate net salary
    net_annual_salary = salary - deductions #Calculates net annual salary by subtracting the final running total of deductions previously calculated
    net_monthly_salary = round(net_annual_salary / 12,2) #Dividing by 12 to get monthly net salary while also rounding the value to 2 decimal places

    return net_monthly_salary #Returns calculated net montly salary to be used in main function



#Function specifically for ROU:
def applying_rou_tax(salary):
    deductions = 0  # Reset deductions for each salary

        # Income Tax:
    taxable_income = salary #Setting annual gross income as taxable income as personal allowance is taken off in first band
    min_income_predicate = [0, 12570, 50270, 125140] #Setting list of all minimumn values to enter each tax band
    max_income_predicate = [12571, 50271, 125141, float('inf')] #Setting list of all max values of each tax band, infinte value is used to compare when the band has no max value set
    tax_rate = [0, 0.2, 0.4, 0.45] #Setting tax rates for each tax band

    for i in range(4): #Looping through the number of tax bands
        if taxable_income > min_income_predicate[i]: #Judging if income in high enough to enter tax band
                
            if taxable_income > max_income_predicate[i]: #Checking if income is higher than the max value of tax band
                deductions += tax_rate[i] * (max_income_predicate[i] - min_income_predicate[i]) #deducitng the tax rate applied the the eligible amount of salary for that band
            else:
                deductions += tax_rate[i] * (taxable_income - min_income_predicate[i]) #When the income maxes out within the band and wont go any bands higher, the total income is used as the max value to apply tax rate as it falls within the band

        # National Insurance:
    taxable_income = salary #All of these have the same logic as the income tax application except the values change 
    min_income_predicate = [0, 11904, 50270]
    max_income_predicate = [11905, 50271, float('inf')]
    tax_rate = [0, 0.08, 0.02]

    for i in range(3):  #Iterates a different amount of times as income tax to match the number of National Insurance bands
        if taxable_income > min_income_predicate[i]:
                
            if taxable_income > max_income_predicate[i]:
                deductions += tax_rate[i] * (max_income_predicate[i] - min_income_predicate[i])
            else:
                deductions += tax_rate[i] * (taxable_income - min_income_predicate[i])

        # Calculate net salary
    net_annual_salary = salary - deductions #Calculates net annual salary by subtracting the final running total of deductions previously calculated
    net_monthly_salary = round(net_annual_salary / 12,2) #Dividing by 12 to get monthly net salary while also rounding the value to 2 decimal places

    return net_monthly_salary #Returns the calculated net monthly salary to be used in main function



applying_tax(df) #Reading dataframe into the function to apply the eligible taxes

(      Monthly Net Income  Annual Gross Income
 0                3267.13             52318.70
 1                1138.54             14139.53
 2                2093.16             30089.14
 3                2376.58             34879.34
 4                1716.62             23764.31
 ...                  ...                  ...
 1013             4933.23             88758.55
 1014             2175.89             31487.39
 1015             2368.52             34743.11
 1016             2305.89             33684.58
 1017             3367.85             54476.98
 
 [1018 rows x 2 columns],
       Monthly Net Income  Annual Gross Income
 0                3213.27             48740.17
 1                2526.40             37292.33
 2                2432.07             35720.17
 3                2432.09             35720.50
 4                4652.77             78154.07
 ...                  ...                  ...
 3977             1770.47             24693.50
 3978             5715.20       

### Functionality

Write a program that converts monthly net income to annual gross income. You can assume that the monthly income is constant month to month. It is *strongly* advised to first write functionality to convert gross to net and confirm this with the provided test set. Your grade for this section will be based solely on your calculations for the holdout set.

For transparency, write your algorithm according to the tax brackets and rates communicated in the presentation slides.

In [ ]:
def calculating_gross_annual_income(salary_df):
    gross_annual_salaries_sco = [] #Initialising lists to store the gross annual salaries once they have been calculated
    gross_annual_salaries_rou = []

    location_salaries = salary_df.groupby('location') #Grouping by location
    rou_data = location_salaries.get_group('rou') #Creating dataframe with rest of UK salaries
    scotland_data = location_salaries.get_group('sco') #Creating dataframe with Scotland salaries

    # Extract monthly net incomes as a list
    net_monthly_incomes_sco = scotland_data['monthly_net_income'].values.tolist()
    net_monthly_incomes_rou = rou_data['monthly_net_income'].values.tolist()

    #For Scottish Data
    for net_montly_income in net_monthly_incomes_sco:
        net_annual = net_montly_income * 12 #Calculating the annual net income
        additions = 0 #Rather than deductions, additions will be used to add back value initially taken from the taxes. It is reset to 0
        
        taxable_income = net_annual #Setting annual gross income as taxable income as personal allowance is taken off in first band
        min_income_predicate = [0, 12570, 14876, 26561, 43662, 75000, 125140] #Setting list of all minimumn values to enter each tax band
        max_income_predicate = [12571, 14877, 26562, 43663, 75001, 125141, float('inf')] #Setting list of all max values of each tax band, infinte value is used to compare when the band has no max value set
        tax_rate = [0, 0.19, 0.2, 0.21, 0.42, 0.45, 0.48] #Setting tax rates for each tax band

        for i in range(7): #Looping through the number of tax bands
            if taxable_income > min_income_predicate[i]: #Judging if income in high enough to enter tax band
                
                if taxable_income > max_income_predicate[i]: #Checking if income is higher than the max value of tax band
                    additions += tax_rate[i] * (max_income_predicate[i] - min_income_predicate[i]) #deducitng the tax rate applied the the eligible amount of salary for that band
                else:
                    additions += tax_rate[i] * (taxable_income - min_income_predicate[i]) #When the income maxes out within the band and wont go any bands higher, the total income is used as the max value to apply tax rate as it falls within the band
                

        # National Insurance:
        taxable_income = net_annual #All of these have the same logic as the income tax application except the values change 
        min_income_predicate = [0, 11904, 50270]
        max_income_predicate = [11905, 50271, float('inf')]
        tax_rate = [0, 0.08, 0.02]

        for i in range(3):  #Iterates a different amount of times as income tax to match the number of National Insurance bands
            if taxable_income > min_income_predicate[i]:
                
                if taxable_income > max_income_predicate[i]:
                    additions += tax_rate[i] * (max_income_predicate[i] - min_income_predicate[i])
                else:
                    additions += tax_rate[i] * (taxable_income - min_income_predicate[i])
                

        gross_annual_income =  round(net_annual + additions,2) #calculating the first estimation of the gross annual income
        estimated_net_monthly_income = applying_scottish_tax(gross_annual_income) #Inputing this estimation into the relevant helper function to see what it caluculates the corresponding monthly net value would be from that gross value
        

        #This code then loops through using the helper function to find the difference from the monthly net value of the gross that was calculated, to the actual value of the monthly net
        #Using this difference, it will multiply it by 12 to find the annual difference and then add that to the calculated gross to improve its closeness to the the actual gross value
        #It will repeat this until the differecne is continuously reduced and the calculated gross value become valuated at what it should be to give the monthly net value that was given
        while estimated_net_monthly_income != net_montly_income:
            estimated_net_monthly_income = applying_scottish_tax(gross_annual_income)
            difference = net_montly_income - estimated_net_monthly_income
            gross_annual_income += (difference * 12)
        gross_annual_salaries_sco.append(round(gross_annual_income,2))


    #For Rest of UK Data - Same code as for Scotland but the income tax bands are ammended
    for net_montly_income in net_monthly_incomes_rou:
        net_annual = net_montly_income * 12
        additions = 0
        
        taxable_income = net_annual 
        min_income_predicate = [0, 12570, 50270, 125140] 
        max_income_predicate = [12571, 50271, 125141, float('inf')] 
        tax_rate = [0, 0.2, 0.4, 0.45] 

        for i in range(4): 
            if taxable_income > min_income_predicate[i]: 
                
                if taxable_income > max_income_predicate[i]: 
                    additions += tax_rate[i] * (max_income_predicate[i] - min_income_predicate[i]) 
                else:
                    additions += tax_rate[i] * (taxable_income - min_income_predicate[i]) 
                

        # National Insurance:
        taxable_income = net_annual  
        min_income_predicate = [0, 11904, 50270]
        max_income_predicate = [11905, 50271, float('inf')]
        tax_rate = [0, 0.08, 0.02]

        for i in range(3):  
            if taxable_income > min_income_predicate[i]:
                
                if taxable_income > max_income_predicate[i]:
                    additions += tax_rate[i] * (max_income_predicate[i] - min_income_predicate[i])
                else:
                    additions += tax_rate[i] * (taxable_income - min_income_predicate[i])
                

        gross_annual_income =  round(net_annual + additions,2) 
        estimated_net_monthly_income = applying_rou_tax(gross_annual_income)
        

        while estimated_net_monthly_income != net_montly_income:
            estimated_net_monthly_income = applying_rou_tax(gross_annual_income)
            difference = net_montly_income - estimated_net_monthly_income
            gross_annual_income += (difference * 12)
        gross_annual_salaries_rou.append(round(gross_annual_income,2))


    #Creating dataframes to display all of the calculated data
    scottish_net_to_gross_incomes = pd.DataFrame({
        'Location': 'Sco',
        'Monthly Net Income': net_monthly_incomes_sco,
        'Gross Annual Income': gross_annual_salaries_sco
    })

    rou_net_to_gross_incomes = pd.DataFrame({
        'Location': 'ROU',
        'Monthly Net Income': net_monthly_incomes_rou,
        'Gross Annual Income': gross_annual_salaries_rou
    })


    #Creating excel file to insert dataframes into
    with pd.ExcelWriter('Salary_Holdout_24_Results.xlsx', engine='openpyxl') as writer:
        scottish_net_to_gross_incomes.to_excel(writer, sheet_name='Scotland', index=False) #Creates and displays data into 2 sheets on the same excel file
        rou_net_to_gross_incomes.to_excel(writer, sheet_name='ROU', index=False)


    #Returning dataframes
    return scottish_net_to_gross_incomes, rou_net_to_gross_incomes

calculating_gross_annual_income(salaries)

(    Location  Monthly Net Income  Gross Annual Income
 0        Sco             2919.71             44226.15
 1        Sco             3905.78             66003.95
 2        Sco             2368.79             34747.75
 3        Sco             2437.59             35910.57
 4        Sco             2042.30             29229.58
 ..       ...                 ...                  ...
 978      Sco             1263.74             16216.22
 979      Sco             1964.62             27916.65
 980      Sco             2886.02             43489.59
 981      Sco              815.16              9781.92
 982      Sco              958.08             11496.96
 
 [983 rows x 3 columns],
      Location  Monthly Net Income  Gross Annual Income
 0         ROU             3041.79             45882.13
 1         ROU             2466.31             36290.84
 2         ROU             2862.15             42888.14
 3         ROU             1601.49             21877.17
 4         ROU             2385.6